In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('stroke_balanced_sample.csv')


target = df['stroke']
features = df.drop(columns=['stroke'])


In [3]:
features_sample = features.head(100)
target_sample = target.head(100)

In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model = "llama3.1:latest")

def predict_stroke_llm(row, llm):
    prompt = f"""
You are a medical AI assistant. Given the following patient data, your task is to predict the likelihood of a stroke. 
Provide your response as either 0 (no stroke) or 1 (stroke), and explain your reasoning based on the data.

Patient data:
- Gender: {row['gender']}
- Age: {row['age']}
- Hypertension: {row['hypertension']}
- Heart Disease: {row['heart_disease']}
- Ever Married: {row['ever_married']}
- Work Type: {row['work_type']}
- Residence Type: {row['Residence_type']}
- Average Glucose Level: {row['avg_glucose_level']}
- BMI: {row['bmi']}
- Smoking Status: {row['smoking_status']}

Prediction:
"""

    response = llm.invoke([
        {"role": "system", "content": "You are a medical expert specialized in stroke prediction."},
        {"role": "user", "content": prompt}
    ])

    output = response.content.strip()

    # Naive binary extraction (adjust based on your LLM's style)
    return int('1' in output.split('\n')[0])


In [6]:
preds = features_sample.apply(lambda row: predict_stroke_llm(row, llm), axis=1)

In [7]:
acc = accuracy_score(target_sample, preds)
print(f"Accuracy: {acc:.2f}")

Accuracy: 0.61
